In [ ]:
import random

# Distance Matrix

In [ ]:
dist_matrix = [
    [0, 31, 32, 5, 30, 9, 15, 40, 14, 21, 7, 13], 
    [31, 0, 5, 30, 40, 15, 8, 2, 4, 17, 50, 27], 
    [32, 5, 0, 32, 25, 16, 3, 3, 10, 8, 40, 21], 
    [5, 30, 32, 0, 50, 6, 30, 53, 10, 35, 12, 20], 
    [30, 40, 25, 50, 0, 32, 10, 20, 34, 7, 20, 6], 
    [9, 15, 16, 6, 32, 0, 15, 15, 4, 14, 21, 25], 
    [15, 8, 3, 30, 10, 15, 0, 6, 9, 4, 9, 9], 
    [40, 2, 3, 53, 20, 15, 6, 0, 7, 8, 30, 29], 
    [14, 4, 10, 10, 34, 4, 9, 7, 0, 13, 31, 35], 
    [21, 17, 8, 35, 7, 14, 4, 8, 13, 0, 12, 11], 
    [7, 50, 40, 12, 20, 21, 9, 30, 31, 12, 0, 5], 
    [13, 27, 21, 20, 6, 25, 9, 29, 35, 11, 5, 0]
]

num_cities = len(dist_matrix)

# Finding the TSP tour

In [ ]:
class individual():
    def __init__(self, tour, length, fitness):
        self.tour = tour
        self.length = length
        self.fitness = fitness

In [ ]:
def tourLength(tour):
    dist = sum([dist_matrix[tour[i]][tour[i+1]] for i in range(len(tour)-1)])
    dist += dist_matrix[tour[-1]][tour[0]]
    return dist

Repeated nearest neighbour algorithm to generate a good initial tour

In [ ]:
def NN(start):
    tour = [start]
    while len(tour) < num_cities:
        nbs = [i for i in range(num_cities) if i not in tour]
        nxt = min(nbs, key=lambda x: dist_matrix[tour[-1]][x])
        tour.append(nxt)
        
    return tour, tourLength(tour)


def repeatedNN():
    tour, tourLength = NN(0)
    for i in range(1, num_cities):
        temp, tempLength = NN(i)
        if tourLength > tempLength:
            tour, tourLength = temp, tempLength
    
    return tour, tourLength

Simulated anneahling algorithm to improve upon the initial tour

In [ ]:
def SA(T_init=1000, repeats=7**6):
    e = 2.718281828459045
    current, currentCost = repeatedNN() 
    T = T_init
    for i in range(repeats):
        T = e**(-T/5)
        a, b = random.sample(range(num_cities), 2)
        current[a], current[b] = current[b], current[a]
        newCost = tourLength(current)
        if newCost < currentCost:
            currentCost = newCost
        else:
            x = random.uniform(0,1)
            if x < e**((currentCost - newCost)/T):
                currentCost = newCost
            else:
                current[a], current[b] = current[b], current[a]

    return current, currentCost

Finally, genetic algorithm to optimize the SA tour

In [ ]:
# Breed two individuals by randomly selecting a part of the parent genes
def breed(a,b):
    child = []
    x, y = random.sample(range(num_cities), 2)
    start, end = min(x, y), max(x, y)
    child = [i for i in a.tour[start:end]]
    child += [i for i in b.tour if i not in child]

    return individual(child, tourLength(child), 0)


# Generate semi random population by breeding the SA tour with itself
def semiRandPop(size):
    pop = []
    popLength = 0
    SATour, SATourLen = SA()
    SATourInd = individual(SATour, SATourLen, 0)
    pop.append(SATourInd)
    for i in range(size-1):
        ind = breed(SATourInd, SATourInd)
        pop.append(ind)
        popLength += ind.length

    return pop, popLength


# Normalise fitness for selection process
def normaliseFitness(pop, popLength):
    a = 0
    for i in pop:
        a += i.length
        i.fitness = i.length/popLength


# Choose an individual from the current population 
def choose(pop):
    a = random.uniform(0, 0.99)
    i = 0
    while a > 0:
        a -= pop[i].fitness
        i += 1
    
    return pop[i-1]


# Randomly mutate a given individual's genes 
def mutate(x, rate=0.2):
    for i in range(num_cities):
        if(random.random() < rate):
            a, b = random.sample(range(num_cities), 2)
            x.tour[a], x.tour[b] = x.tour[b], x.tour[a]   
        
    x.length = tourLength(x.tour)
    return x


def geneticAlg(pSize=100, numIter=500):
    pop, popLength = semiRandPop(pSize)
    fittest = min(pop, key=lambda x: x.length)
    for _ in range(numIter):
        newPop = []
        normaliseFitness(pop, popLength)
        for i in range(pSize): 
            indX, indY = choose(pop), choose(pop)
            indZ = breed(indX, indY)
            indZ = mutate(indZ)
            fittest = indZ if indZ.length < fittest.length else fittest
            newPop.append(indZ)

        pop += newPop
        pop.sort(key=lambda x: x.length)
        pop = pop[:pSize]
        popLength = sum([i.length for i in pop])
    
    return fittest

In [ ]:
fittest = geneticAlg()
fittest.tour, fittest.length

([0, 3, 5, 8, 1, 7, 2, 6, 9, 4, 11, 10], 56)